In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale,PolynomialFeatures
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
from mlxtend.regressor import StackingRegressor
from collections import defaultdict

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
data_set1 = pd.read_excel('../复赛数据/训练_20180117.xlsx')
data_set2 = pd.read_excel('../复赛数据/测试A_20180117.xlsx')
y2 = pd.read_csv('../复赛数据/[new] fusai_answer_a_20180127.csv', names=['ID', 'Value'])


submit = pd.read_csv('../复赛数据/answer_sample_b_20180117.csv', names=['id', 'Y'])
test_set = pd.read_excel('../复赛数据/测试B_20180117.xlsx')

data_set2 = pd.concat([data_set2, y2[['Value']]], axis=1)
data_set = pd.concat([data_set1, data_set2], axis=0)


y = data_set[['Value']]
y.reset_index(inplace=True, drop=['index'])
data_set = data_set.drop(['Value'], axis=1)
Le = LabelEncoder()

In [4]:
input_data = pd.concat([data_set, test_set], ignore_index=True)
input_data = input_data.drop(['ID'], axis=1)

In [5]:
col = pd.DataFrame(input_data.ix[:, 2:].columns, columns=['col'])
TOOL_ID_col_dict = defaultdict(lambda : 0)
for index,row in col.iterrows():
    info=row.col.split('X')[0]
    TOOL_ID_col_dict[info] += 1

TOOL_ID_col_dict

defaultdict(<function __main__.<lambda>>,
            {'210': 228,
             '220': 179,
             '300': 21,
             '310': 170,
             '311': 184,
             '312': 679,
             '330': 889,
             '340': 139,
             '344': 398,
             '360': 1030,
             '400': 183,
             '420': 173,
             '440A': 213,
             '520': 382,
             '750': 1030,
             'Chamber': 1,
             'Chamber ID': 1,
             'ERROR:#N/A': 1,
             'ERROR:#N/A (#1)': 1,
             'ERROR:#N/A (#2)': 1,
             'ERROR:#N/A (#3)': 1,
             'ERROR:#N/A_1': 1,
             'ERROR:#N/A_1 (#1)': 1,
             'ERROR:#N/A_1 (#2)': 1,
             'ERROR:#N/A_10': 1,
             'ERROR:#N/A_11': 1,
             'ERROR:#N/A_12': 1,
             'ERROR:#N/A_13': 1,
             'ERROR:#N/A_14': 1,
             'ERROR:#N/A_15': 1,
             'ERROR:#N/A_16': 1,
             'ERROR:#N/A_17': 1,
             'ERROR

In [6]:
#删除全空字段
null_col = []
for col in data_set.columns:
    if data_set[col].isnull().all() == True:
        null_col.append(col)
        
input_data.drop(null_col, axis=1, inplace=True)    
input_data.drop(['220X150', '220X151'], axis=1, inplace=True)    

In [7]:
#转换obj字段
obj_col = []
for col in input_data.columns:
    if input_data[col].dtypes == object:
        obj_col.append(col)
        
for i in range(len(obj_col)):
    Le.fit(input_data[obj_col[i]].unique())
    input_data[obj_col[i]] = Le.transform(input_data[obj_col[i]])

In [8]:
#删除较大值
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

input_data = date_cols(input_data)

In [9]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.mean())      
    for line in data.columns:
        if len(data[line].unique()) == 1:
            data = data.drop([line], axis=1)    
    return data

input_data = drop_col(input_data)

In [10]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 3 * (Q3 - Q1)
        max_v = Q3 + 3 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].mean()
           
    return data

#异常值处理完再清晰一遍数据
input_data = quantile_dropout(input_data)
input_data.fillna(input_data.mean(), inplace=True)
input_data = drop_col(input_data)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
#剔除小方差字段
VT=VarianceThreshold(threshold=0.5)
input_data = pd.DataFrame(VT.fit_transform(input_data))

In [12]:
train = input_data.ix[:1099, :]
test = input_data.ix[1100:, :]
clf_gt = ensemble.GradientBoostingRegressor(learning_rate=0.01, n_estimators=200, random_state=0)
clf_gt.fit(train, y)

model = SelectFromModel(clf_gt, prefit=True) 
train = pd.DataFrame(model.transform(train))
test = pd.DataFrame(model.transform(test))

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
#单xgb模型线下
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.6
param['seed'] = 0
xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_boost_round=9800, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0]) 

0.0105656


In [14]:
#单xgb模型线上
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.6
param['seed'] = 0
xgbTrain = xgb.DMatrix(train, label=y)
xgbTest = xgb.DMatrix(test)
modle = xgb.train(param, xgbTrain, num_boost_round=9200)

In [ ]:
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, 
                                              max_depth=2, 
                                              subsample=0.8, 
                                              learning_rate=0.01, 
                                              random_state=2, 
                                              max_features=0.2)
modle0  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.6, 
                           subsample=0.8, 
                           seed=2, 
                           n_estimators=8800)
modle1  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.6, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=8800)

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3,
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=1200, 
                         seed=2)
base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['lgb', clf1],]


folds = list(KFold(len(train), n_folds=5, random_state=0))
S_train = np.zeros((train.shape[0], len(base_model)))
S_test = np.zeros((test.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((test.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = train.ix[train_idx, :]
        X_valid = train.ix[test_idx, :]
        Y = y.ix[train_idx, :]
        clf.fit(X_train, Y['Value'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(test)       
    S_test[:, index] = S_test_i.mean(1)
    
linreg = linear_model.LinearRegression()
linreg.fit(S_train, y)
result = linreg.predict(S_test)
print('Done')

the model 0


In [46]:
fin = pd.DataFrame(result, columns=['stacking'])
fin['singal'] = modle.predict(xgbTest)
fin['mean'] = fin.mean(axis=1)

,stacking,singal,mean
0,2.912807,2.889987,2.901397
1,3.007286,2.964874,2.986080
2,2.685882,2.662324,2.674103
3,2.806926,2.759750,2.783338
4,2.652038,2.619831,2.635935
5,2.787010,2.722117,2.754563
6,2.652562,2.627702,2.640132
7,2.588135,2.463275,2.525705
8,3.031880,2.980239,3.006060
9,2.666783,2.645448,2.656115


In [ ]:
submit['Y'] = fin['mean'].values 

In [ ]:
submit.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)